In [84]:
#here we are trying to apply different classification algos, on pca transformed data on lfw dataset and check accuracy
#in each case.
#different classification algos:
#logisticRegression
#naive bayes
#random forests
#decision trees
#svm
#knn

In [85]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV    #for finding best combination of paramenters

In [86]:
lfw=datasets.fetch_lfw_people(min_faces_per_person=100,resize=0.4)

In [87]:
lfw.keys()

dict_keys(['data', 'images', 'target', 'target_names', 'DESCR'])

In [88]:
#since these are images ,no need to do feaure scaling before.


In [89]:
X=lfw.data

In [90]:
Y=lfw.target

In [91]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=0)

In [92]:
pca=PCA()   #initiallly not passing n_componets =k, as we are still trying best k.

In [93]:
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [94]:
pca.explained_variance_

array([4.83559188e+05, 4.21049406e+05, 1.85467703e+05, 1.52713578e+05,
       1.37901984e+05, 7.91334453e+04, 6.65560312e+04, 5.67629336e+04,
       5.26333125e+04, 4.91668320e+04, 3.97866172e+04, 3.67396445e+04,
       3.45940547e+04, 2.78019434e+04, 2.61085293e+04, 2.47647168e+04,
       2.34815566e+04, 2.24296934e+04, 2.12212246e+04, 1.89539688e+04,
       1.78013867e+04, 1.75217188e+04, 1.58070215e+04, 1.47426602e+04,
       1.41662578e+04, 1.36058574e+04, 1.29047188e+04, 1.21215117e+04,
       1.17997344e+04, 1.13332471e+04, 1.01976494e+04, 9.76716797e+03,
       9.52379492e+03, 9.24495215e+03, 8.61826172e+03, 8.53404785e+03,
       8.34165234e+03, 8.06421777e+03, 7.86919824e+03, 7.57776123e+03,
       7.45386768e+03, 7.25952637e+03, 6.79561768e+03, 6.56742432e+03,
       6.50352832e+03, 6.35414355e+03, 6.05674805e+03, 5.88387451e+03,
       5.77712842e+03, 5.54030957e+03, 5.28675342e+03, 5.21347559e+03,
       5.03800879e+03, 4.89313037e+03, 4.87861719e+03, 4.71101416e+03,
      

In [95]:
#finding best k for this training data

In [96]:
k=0
total=sum(pca.explained_variance_)
currentVarianceSum=0
while currentVarianceSum/total<0.99:
    currentVarianceSum+=pca.explained_variance_[k]
    k+=1

In [97]:
k          #so best k=316, for keeping 99% variance

316

In [98]:
x_train.shape     #initially 1850 features

(855, 1850)

In [99]:
pca=PCA(n_components=316,whiten=True) 

In [100]:
x_train_pca=pca.fit_transform(x_train)
x_test_pca=pca.transform(x_test)

In [101]:
x_train_pca.shape,x_test_pca.shape       #features reduced to 316

((855, 316), (285, 316))

In [102]:
#now applying various classification algos

In [103]:
#1. Logistic regression classifier.
lr=LogisticRegression()

In [104]:
lr.fit(x_train_pca,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [105]:
y_pred=lr.predict(x_test_pca)

In [106]:
lr.score(x_test_pca,y_test)

0.8280701754385965

In [107]:
print(confusion_matrix(y_true=y_test,y_pred=y_pred))

[[ 52   2   4   1   2]
 [  2  28   5   1   0]
 [  4   5 109   4   0]
 [  4   1   6  20   0]
 [  1   0   6   1  27]]


In [109]:
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84        61
           1       0.78      0.78      0.78        36
           2       0.84      0.89      0.87       122
           3       0.74      0.65      0.69        31
           4       0.93      0.77      0.84        35

   micro avg       0.83      0.83      0.83       285
   macro avg       0.82      0.79      0.80       285
weighted avg       0.83      0.83      0.83       285



In [110]:
grid={"C":[1e0,1e2,1e3,1e4,5e3,5e4]}

In [111]:
abc=GridSearchCV(lr,grid)

In [112]:
abc.fit(x_train_pca,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWar

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1.0, 100.0, 1000.0, 10000.0, 5000.0, 50000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [113]:
abc.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [114]:
#c=1.0 (best) by grid search. So keep it as it is.

In [34]:
#2.Naive bayes classifier: gaussian and multinomial

In [35]:
gnb=GaussianNB()

In [36]:
gnb.fit(x_train_pca,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [37]:
y_pred=gnb.predict(x_test_pca)

In [38]:
gnb.score(x_test_pca,y_test)

0.6140350877192983

In [41]:
print('gaussian nb:')
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

gaussian nb:
              precision    recall  f1-score   support

           0       0.46      0.72      0.56        61
           1       0.63      0.67      0.65        36
           2       0.68      0.69      0.69       122
           3       0.89      0.26      0.40        31
           4       0.75      0.43      0.55        35

   micro avg       0.61      0.61      0.61       285
   macro avg       0.68      0.55      0.57       285
weighted avg       0.66      0.61      0.61       285

[[44  2 14  1  0]
 [ 8 24  4  0  0]
 [29  7 84  0  2]
 [ 8  2 10  8  3]
 [ 6  3 11  0 15]]


In [42]:
mnb=MultinomialNB()

In [44]:
#mnb.fit(x_train_pca,y_train) : didn't work

In [45]:
#3. randomForest

In [46]:
rf=RandomForestClassifier()

In [47]:
rf.fit(x_train_pca,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
y_pred=rf.predict(x_test_pca)

In [49]:
rf.score(x_test_pca,y_test)

0.47368421052631576

In [50]:
print('Random forest:')
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

Random forest:
              precision    recall  f1-score   support

           0       0.47      0.44      0.46        61
           1       0.18      0.06      0.09        36
           2       0.52      0.84      0.64       122
           3       0.25      0.06      0.10        31
           4       0.11      0.03      0.05        35

   micro avg       0.47      0.47      0.47       285
   macro avg       0.31      0.29      0.27       285
weighted avg       0.39      0.47      0.40       285

[[ 27   1  29   1   3]
 [  9   2  22   2   1]
 [ 10   5 103   2   2]
 [  1   1  25   2   2]
 [ 10   2  21   1   1]]


In [51]:
#4. decision trees


In [52]:
dtree=DecisionTreeClassifier()

In [54]:
dtree.fit(x_train_pca,y_train)
y_pred=dtree.predict(x_test_pca)

In [55]:
dtree.score(x_test_pca,y_test)

0.47719298245614034

In [56]:
print('Decision tree:')
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

Decision tree:
              precision    recall  f1-score   support

           0       0.52      0.44      0.48        61
           1       0.22      0.19      0.21        36
           2       0.59      0.69      0.63       122
           3       0.32      0.26      0.29        31
           4       0.30      0.29      0.29        35

   micro avg       0.48      0.48      0.48       285
   macro avg       0.39      0.37      0.38       285
weighted avg       0.46      0.48      0.47       285

[[27  6 19  3  6]
 [ 9  7 14  2  4]
 [10 12 84  6 10]
 [ 3  6 11  8  3]
 [ 3  1 15  6 10]]


In [58]:
#4.knn


In [115]:
knn=KNeighborsClassifier()     #using grid search to find best k

In [117]:
grid={"n_neighbors":[1,3,5,7,9,11]}

In [118]:
abc=GridSearchCV(knn,grid)

In [119]:
abc.fit(x_train_pca,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [120]:
abc.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [121]:
#k or n_neighbors=3 (best), #so use k=3

In [122]:
knn=KNeighborsClassifier(n_neighbors=3)

In [123]:
knn.fit(x_train_pca,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [124]:
y_pred=knn.predict(x_test_pca)

In [125]:
knn.score(x_test_pca,y_test)

0.5614035087719298

In [126]:
print('KNN:')
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

KNN:
              precision    recall  f1-score   support

           0       0.93      0.44      0.60        61
           1       0.88      0.19      0.32        36
           2       0.52      0.93      0.66       122
           3       1.00      0.03      0.06        31
           4       0.41      0.34      0.38        35

   micro avg       0.56      0.56      0.56       285
   macro avg       0.75      0.39      0.40       285
weighted avg       0.69      0.56      0.51       285

[[ 27   0  30   0   4]
 [  1   7  27   0   1]
 [  1   0 113   0   8]
 [  0   1  25   1   4]
 [  0   0  23   0  12]]


In [57]:
#4.svm: linear kernel, linear svm, gaussian svm, polynomial svm

In [127]:
#since svm also support multiclass , we can work on all features instead of just 2.

In [128]:
clf=svm.SVC(kernel='linear')

In [129]:
grid={"C":[1e0,1e2,1e3,1e4,5e3,5e2]}

In [130]:
abc=GridSearchCV(clf,grid)

In [131]:
abc.fit(x_train_pca,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1.0, 100.0, 1000.0, 10000.0, 5000.0, 500.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [132]:
abc.best_estimator_

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [133]:
#C=1.0 (best) for kernel=linear, so now using it

In [134]:
clf=svm.SVC(kernel='linear',C=1.0)

In [135]:
clf.fit(x_train_pca,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [136]:
clf.score(x_test_pca,y_test)

0.8421052631578947

In [137]:
y_pred=clf.predict(x_test_pca)

In [138]:
print('SVM with linear kernel:')
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

SVM with linear kernel:
              precision    recall  f1-score   support

           0       0.78      0.85      0.81        61
           1       0.81      0.72      0.76        36
           2       0.86      0.92      0.89       122
           3       0.88      0.71      0.79        31
           4       0.90      0.80      0.85        35

   micro avg       0.84      0.84      0.84       285
   macro avg       0.85      0.80      0.82       285
weighted avg       0.84      0.84      0.84       285

[[ 52   1   4   1   3]
 [  5  26   5   0   0]
 [  5   3 112   2   0]
 [  3   2   4  22   0]
 [  2   0   5   0  28]]


In [139]:
#svm with gaussian kernel

In [141]:
clf=svm.SVC()

In [142]:
grid={"C":[1,1e2,1e3,1e4,5e2,5e3],    #here parameters are C and gamma ,so used them as key in dictionary
      "gamma":[1e-2,1e-3,1e-4,5e-2,5e-3,5e-4]}

In [143]:
abc=GridSearchCV(clf,grid)

In [144]:
abc.fit(x_train_pca,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 100.0, 1000.0, 10000.0, 500.0, 5000.0], 'gamma': [0.01, 0.001, 0.0001, 0.05, 0.005, 0.0005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [145]:
abc.best_estimator_

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
#C=100, gamma=0.001 (best) and best kernel to use is rbf, i.e gaussian

In [146]:
clf=svm.SVC(kernel='rbf',C=100.0,gamma=0.001)

In [147]:
clf.fit(x_train_pca,y_train)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [148]:
y_pred=clf.predict(x_test_pca)

In [149]:
clf.score(x_test_pca,y_test)

0.8736842105263158

In [150]:
print('SVM with rbf kernel:')
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

SVM with rbf kernel:
              precision    recall  f1-score   support

           0       0.84      0.92      0.88        61
           1       0.90      0.72      0.80        36
           2       0.86      0.94      0.90       122
           3       0.96      0.74      0.84        31
           4       0.94      0.83      0.88        35

   micro avg       0.87      0.87      0.87       285
   macro avg       0.90      0.83      0.86       285
weighted avg       0.88      0.87      0.87       285

[[ 56   1   1   1   2]
 [  2  26   8   0   0]
 [  6   1 115   0   0]
 [  2   1   5  23   0]
 [  1   0   5   0  29]]
